In [122]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import scipy.stats as stats
from geopy.geocoders import Nominatim
import time

In [224]:
geolocator = Nominatim(user_agent="grace")
location = geolocator.reverse("40.84916900104506, -73.86835600032798")
print(location.address)

Bronx Park East, Birchall Avenue, Bronxdale, The Bronx, Bronx County, NYC, New York, 10462, USA


In [169]:
import uszipcode
from uszipcode import Zipcode
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)

In [234]:
def coord_to_zip(lat, long, radius=3, returns=1):
    result = search.by_coordinates(float(lat), float(long), radius=radius, returns=returns)
    try:
        return result[0].zipcode
    except:
        return "No zipcode found"

In [164]:
def address_from_coord(lat, long):
    start = time.time()
    geolocator = Nominatim(user_agent="jdmitchell")
    location = geolocator.reverse(f'{lat}, {long}')
    if time.time()-start<1:
        time.sleep(1.01-(time.time()-start))
    return location.address

In [165]:
def zipcode_from_coord(lat, long):
    return address_from_coord(lat, long).split(", ")[-2]

In [237]:
address_from_coord(40.84916900104506, -73.86835600032798)

'Bronx Park East, Birchall Avenue, Bronxdale, The Bronx, Bronx County, NYC, New York, 10462, USA'

In [137]:
zipcode_from_coord(40.84916900104506, -73.86835600032798)

'10462'

In [13]:
df_subway = pd.read_csv("subway_entrances.csv")

In [82]:
df_subway['latlong']=df_subway['the_geom'].str[7:-1]

In [83]:
df_subway[df_subway.latlong == '-73.86835600032798 40.84916900104506']

,OBJECTID,URL,NAME,the_geom,LINE,latlong,lat,long
0,1734,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NW corner,POINT (-73.86835600032798 40.84916900104506),2-5,-73.86835600032798 40.84916900104506,40.84916900104506,-73.86835600032798


In [115]:
df_exerpt_subway = df_subway[:10]

In [121]:
df_exerpt_subway

,OBJECTID,URL,NAME,the_geom,LINE,latlong,lat,long,zipcode
0,1734,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NW corner,POINT (-73.86835600032798 40.84916900104506),2-5,"[-73.86835600032798, 40.84916900104506]",40.84916900104506,-73.86835600032798,10462
1,1735,http://web.mta.info/nyct/service/,Birchall Ave & Sagamore St at NE corner,POINT (-73.86821300022677 40.84912800131844),2-5,"[-73.86821300022677, 40.84912800131844]",40.84912800131844,-73.86821300022677,10462
2,1736,http://web.mta.info/nyct/service/,Morris Park Ave & 180th St at NW corner,POINT (-73.87349900050798 40.84122300105249),2-5,"[-73.87349900050798, 40.84122300105249]",40.84122300105249,-73.87349900050798,10460
3,1737,http://web.mta.info/nyct/service/,Morris Park Ave & 180th St at NW corner,POINT (-73.8728919997833 40.84145300067447),2-5,"[-73.8728919997833, 40.84145300067447]",40.84145300067447,-73.8728919997833,10460
4,1738,http://web.mta.info/nyct/service/,Boston Rd & 178th St at SW corner,POINT (-73.87962300013866 40.84081500075867),2-5,"[-73.87962300013866, 40.84081500075867]",40.84081500075867,-73.87962300013866,10460
5,1739,http://web.mta.info/nyct/service/,Boston Rd & E Tremont Ave at NW corner,POINT (-73.88000500027815 40.840434000875874),2-5,"[-73.88000500027815, 40.840434000875874]",40.840434000875874,-73.88000500027815,10460
6,1740,http://web.mta.info/nyct/service/,Boston Rd & E Tremont Ave at NE corner,POINT (-73.87983300021861 40.84035400111976),2-5,"[-73.87983300021861, 40.84035400111976]",40.84035400111976,-73.87983300021861,10460
7,1741,http://web.mta.info/nyct/service/,Boston Rd & 178th St at SE corner,POINT (-73.8795549998336 40.84063900116436),2-5,"[-73.8795549998336, 40.84063900116436]",40.84063900116436,-73.8795549998336,10460
8,1742,http://web.mta.info/nyct/service/,Boston Rd & 178th St at NW corner,POINT (-73.87939700013239 40.84107800066419),2-5,"[-73.87939700013239, 40.84107800066419]",40.84107800066419,-73.87939700013239,10460
9,1743,http://web.mta.info/nyct/service/,Boston Rd & 174th St at SW corner,POINT (-73.88804799985908 40.83732500129732),2-5,"[-73.88804799985908, 40.83732500129732]",40.83732500129732,-73.88804799985908,10460


In [119]:
df_exerpt_subway['zipcode'] = df_exerpt_subway.latlong.apply(lambda x: zipcode_from_coord(x[1], x[0]))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [85]:
df_subway['latlong'] = df_subway.latlong.str.split(' ')

In [86]:
df_subway['lat'] = df_subway['latlong'].apply(lambda x: x[1])

In [87]:
df_subway['long'] = df_subway['latlong'].apply(lambda x: x[0])

In [222]:
df_subway['zipcode'] = df_subway.latlong.apply(lambda x: coord_to_zip(x[1], x[0]))
#df_subway['zipcode'] = df_subway.iloc[1:10].latlong.apply(lambda x: coord_to_zip(x[1], x[0]))
#make sure it's not deleting everything besides the section currently being updated

In [230]:
df_subway.zipcode.nunique()

136

In [236]:
coord_to_zip(40.81958300055107,-73.90195799960925)

'10455'

In [247]:
df_test=df_subway.groupby('zipcode').count()

In [259]:
df_test.reset_index()

,zipcode,OBJECTID,URL,NAME,the_geom,LINE,latlong,lat,long
0,10001,51,51,50,51,51,51,51,51
1,10002,19,19,19,19,19,19,19,19
2,10003,28,28,28,28,28,28,28,28
3,10004,6,6,0,6,6,6,6,6
4,10006,30,30,25,30,30,30,30,30
5,10007,26,26,26,26,26,26,26,26
6,10009,4,4,4,4,4,4,4,4
7,10011,72,72,72,72,72,72,72,72
8,10012,34,34,31,34,34,34,34,34
9,10013,14,14,14,14,14,14,14,14


In [244]:
df_subway[df_subway.zipcode == '10001'].shape

(51, 9)

In [260]:
df_subway_count = df_test.reset_index()[['zipcode', 'OBJECTID']]

In [266]:
df_subway_count.rename(columns={'OBJECTID':'num_subway_entrances'},inplace=True)

In [270]:
df_subway_count.num_subway_entrances.unique()

array([51, 19, 28,  6, 30, 26,  4, 72, 34, 14,  5, 16, 27, 12,  3,  9, 10,
       17, 23, 24,  8, 18,  1,  7, 11, 33, 20, 21, 15, 44, 13, 41, 31, 29,
        2, 22])

In [271]:
df_subway_count

,zipcode,num_subway_entrances
0,10001,51
1,10002,19
2,10003,28
3,10004,6
4,10006,30
5,10007,26
6,10009,4
7,10011,72
8,10012,34
9,10013,14
